# Run Test Method

In [8]:
def runTests(testClass):
    loader = unittest.TestLoader()
    suite = loader.loadTestsFromTestCase(testClass)
    runner = unittest.TextTestRunner()
    runner.run(suite)

# Modelo

In [9]:
from __future__ import annotations
import ast, sys
from dataclasses import dataclass
from typing import List, Iterable, Tuple, Optional

@dataclass(frozen=True)
class CodeWarning:
    message: str
    lineno: int
    col: int
    end_lineno: Optional[int] = None
    end_col: Optional[int] = None

class DetectionRule:
    name: str = "Generic Detector"
    def search(self, src: str) -> List[CodeWarning]:
        raise NotImplementedError

class Patcher:
    name: str = "Generic Patcher"
    def patch(self, tree: ast.Node) -> str:
        raise NotImplementedError

class DCCLinter:
    def __init__(self, detectors: Iterable[DetectionRule], patchers: Iterable[Patcher]):
        self.detectors = detectors
        self.patchers = patchers

    def search(self, src: str) -> List[CodeWarning]:
        out: List[CodeWarning] = []
        for d in self.detectors:
            out.extend(d.search(src))
        return sorted(out, key=lambda w: (w.lineno, w.col))

    def patch(self, src: str) -> str:
        tree = ast.parse(src)
        for p in self.patchers:
          tree = p.patch(tree)
        return ast.unparse(tree)

# None Compares

## Detector

In [10]:
class PercentFormatDetectionRule(DetectionRule):
    name = "Detect simple '%' formatting with names"

    def search(self, src: str) -> List[CodeWarning]:
        # Marca cadenas formateadas con %.
        tree = ast.parse(src)
        warnings: List[CodeWarning] = []
        for node in ast.walk(tree):
            if isinstance(node, ast.BinOp) and isinstance(node.op, ast.Mod) and isinstance(node.left, ast.Constant) and isinstance(node.left.value, str):
                warnings.append(CodeWarning("Evita el uso de % para formatear strings, usa f-strings", node.lineno, node.col_offset))
        return warnings

## Patcher

In [11]:
class PercentFormatPatcher(Patcher):
    name = "Fix simple '%' → f-string"

    def patch(self, tree: ast.AST) -> ast.AST:
        # Reescribe '%'-format a f-strings simples.
        class T(ast.NodeTransformer):
            def visit_BinOp(self, node):
                if isinstance(node, ast.BinOp) and isinstance(node.op, ast.Mod) and isinstance(node.left, ast.Constant) and isinstance(node.left.value, str):
                    fmt = node.left.value
                    values = node.right
                    args = values.elts if isinstance(values, ast.Tuple) else [values]
                    parts, first = [], True
                    for chunk in fmt.split('%'):
                        if first:
                            parts.append(chunk); first = False; continue
                        if args:
                            val = args.pop(0)
                            parts.append('{' + ast.unparse(val) + '}')
                        if chunk and chunk[:1] in ('s','d'):
                            parts.append(chunk[1:])
                        else:
                            parts.append(chunk)
                    return ast.parse(f"f'''{''.join(parts)}'''", mode='eval').body
                return self.generic_visit(node)
        return T().visit(tree)

## Tests

In [12]:
import unittest

class TestPercentToFstring(unittest.TestCase):
    def setUp(self):
        self.l = DCCLinter([PercentFormatDetectionRule()], [PercentFormatPatcher()])

    def test_detect(self):
        s = "name='Ana'\nprint('Hi %s' % name)\n"
        self.assertEqual(len(self.l.search(s)), 1)

    def test_patch_single(self):
        s = "name='Ana'\nprint('Hi %s' % name)\n"
        out = self.l.patch(s)
        self.assertIn("print(f'Hi {name}')", out)

    def test_patch_tuple(self):
        s = "a,b=1,2\nprint('%s-%d' % (a,b))\n"
        out = self.l.patch(s)
        self.assertIn("print(f'{a}-{b}')", out)

    # Tests extra
    def test_no_warning_for_fstring(self):
        s = "name='Ana'\nprint(f'Hi {name}')\n"
        self.assertEqual(len(self.l.search(s)), 0)

    def test_patch_mixed_types(self):
        s = "x=3\ny='A'\nprint('%s:%d' % (y, x))\n"
        out = self.l.patch(s)
        self.assertIn("print(f'{y}:{x}')", out)

    def test_no_warning_non_string_left(self):
        s = "a=5\nb=a%2\n"
        self.assertEqual(len(self.l.search(s)), 0)

In [13]:
runTests(TestPercentToFstring)

......
----------------------------------------------------------------------
Ran 6 tests in 0.005s

OK
.....
----------------------------------------------------------------------
Ran 6 tests in 0.005s

OK


## Real World Code

In [14]:
def find_cell(comment):
    """
    Busca la primera celda en el cuaderno actual cuya primera línea
    comience con el comentario dado (por ejemplo, "# Case Study 1").

    Parámetros:
        comment (str): El comentario que se quiere buscar.

    Retorna:
        str | None: El código fuente de la celda encontrada, o None si no existe.
    """
    for cell_source in In:  # `In` contiene el código fuente de todas las celdas ejecutadas
        lines = cell_source.strip().splitlines()
        if lines and lines[0].strip().startswith(comment):  # revisa la primera línea
            return cell_source
    return None

In [15]:
# Case Study 1
user='Ana'
print('%s-%d' % (user, 3))
msg = 'Hi %s' % user

# Dentro de un if
if True:
    info = 'User: %s' % user

# En un bucle
items = []
for i in range(3):
    items.append('%s-%d' % (user, i))

# Dentro de una función
def greet(name, n):
    return 'Hi %s x%d' % (name, n)


Ana-3


In [16]:
linter = DCCLinter([PercentFormatDetectionRule()], [PercentFormatPatcher()])

bad_code =  find_cell("# Case Study 1")
warnings = linter.search(bad_code)
print("-= Warnings =-")
for w in warnings:
    print(w)
good_code = linter.patch(bad_code)
print("-= Fixed Code =-")
print(good_code)

-= Warnings =-
CodeWarning(message='Evita el uso de % para formatear strings, usa f-strings', lineno=3, col=6, end_lineno=None, end_col=None)
CodeWarning(message='Evita el uso de % para formatear strings, usa f-strings', lineno=4, col=6, end_lineno=None, end_col=None)
CodeWarning(message='Evita el uso de % para formatear strings, usa f-strings', lineno=8, col=11, end_lineno=None, end_col=None)
CodeWarning(message='Evita el uso de % para formatear strings, usa f-strings', lineno=13, col=17, end_lineno=None, end_col=None)
CodeWarning(message='Evita el uso de % para formatear strings, usa f-strings', lineno=17, col=11, end_lineno=None, end_col=None)
-= Fixed Code =-
user = 'Ana'
print(f'{user}-{3}')
msg = f'Hi {user}'
if True:
    info = f'User: {user}'
items = []
for i in range(3):
    items.append(f'{user}-{i}')

def greet(name, n):
    return f'Hi {name} x{n}'
